In [ ]:
!pip install skorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
import datetime
import itertools
import numpy as np
import os
import pandas as pd
import patsy
import time

import sklearn
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch import optim
from torch.autograd import Variable

from skorch import NeuralNet
from skorch.helper import predefined_split, SliceDataset
from skorch.callbacks import BatchScoring, Checkpoint, EarlyStopping, EpochScoring, LRScheduler, TensorBoard, ProgressBar

import tensorflow as tf
from tensorflow import summary
#%load_ext tensorboard

In [ ]:
# Torch parameters
identifier = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(identifier)
print(device)

cpu


In [ ]:
if not os.path.exists('ml-100k'):
    !wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
    !unzip -o ml-100k.zip

--2024-01-04 11:41:43--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  16.5MB/s    in 0.3s    

2024-01-04 11:41:44 (16.5 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

### Dataset

In [ ]:
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "FilmNoir", "Horror",
    "Musical", "Mystery", "Romance", "SciFi", "Thriller", "War", "Western"
]

In [ ]:
class rsdataset(Dataset):
    def __init__(self, usersfile, moviesfile, ratingsfile, nrows=None):

        # Read files
        self.movies = pd.read_csv(moviesfile, sep='|', names=['MovieID', 'Title', 'date', 'video_rl_date', 'link']+genre_cols, engine='python', encoding='latin-1')
        self.users = pd.read_csv(usersfile, sep='|', names=['UserID', 'Age', 'Gender', 'Occupation', 'Zipcode'], engine='python', encoding='latin-1')
        self.ratings = pd.read_csv(ratingsfile, sep='\t', names=['UserID', 'MovieID', 'Rating', 'Timestamp'], engine='python', nrows=nrows, encoding='latin-1')

        df2 = self.movies[genre_cols]
        df2['Genre'] = df2.apply(lambda row: ', '.join(row.index[row == 1]), axis=1)
        self.movies['Genre'] = df2['Genre']
        self.movies = self.movies.drop(genre_cols, axis = 1)
        # self.movies['Genre'] = self.movies['Genre'].map(genre_dict)
        bins = [0, 18, 25, 35, 45, 50, 56, 100]
        labels = [1, 18, 25, 35, 45, 50, 56]
        self.users['Age'] = pd.cut(self.users['Age'], bins=bins, labels=labels, right=False)
        assert self.users['UserID'].nunique() >= self.ratings['UserID'].nunique(), 'UserID with unknown information'
        assert self.movies['MovieID'].nunique() >= self.ratings['MovieID'].nunique(), 'Movies with unknown information'

        self.users_emb_columns = []
        self.users_ohe_columns = []
        self.movies_emb_columns = []
        self.movies_ohe_columns = []
        self.interact_columns = []

        self.movies = self.movies.drop(['date', 'video_rl_date', 'link'], axis= 1)
        self.nusers = self.ratings['UserID'].nunique()
        self.nmovies = self.ratings['MovieID'].nunique()

        self.y_range = (self.ratings['Rating'].min(), self.ratings['Rating'].max())

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):

        return (((self.users_emb[idx])),
                ((self.users_ohe[idx])),
                ((self.movies_emb[idx])),
                ((self.movies_ohe[idx])),
                ((self.interact[idx]))), (self.y[idx])

    def to_tensor(self):
        self.users_emb = torch.from_numpy(self.ratings[self.users_emb_columns].values)
        self.users_ohe = torch.tensor(self.ratings[self.users_ohe_columns].values, dtype=torch.float)
        self.movies_emb = torch.from_numpy(self.ratings[self.movies_emb_columns].values)
        self.movies_ohe = torch.tensor(self.ratings[self.movies_ohe_columns].values, dtype=torch.float)
        self.interact = torch.from_numpy(self.ratings[self.interact_columns].values)
        self.y = torch.tensor(self.y.values, dtype=torch.float)

In [ ]:
train = rsdataset('ml-100k/u.user', 'ml-100k/u.item', 'ml-100k/u.data', nrows=None)

<ipython-input-9-f5a438cfccf0>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Genre'] = df2.apply(lambda row: ', '.join(row.index[row == 1]), axis=1)


### Preprocessing of dataset

In [ ]:
train.ratings = train.ratings.merge(train.movies, left_on='MovieID', right_on='MovieID')
train.movies = train.ratings[train.movies.columns]

train.ratings = train.ratings.merge(train.users, left_on='UserID', right_on='UserID')
train.users = train.ratings[train.users.columns]

train.y = train.ratings['Rating']

In [ ]:
# Label Encode users
columns = ['UserID', 'Gender', 'Age', 'Occupation']
train.ratings[columns] = train.ratings[columns].apply(preprocessing.LabelEncoder().fit_transform)
train.users_emb_columns = train.users_emb_columns + columns

In [ ]:
# Label Encode movies
columns = ['MovieID']
train.ratings[columns] = train.ratings[columns].apply(preprocessing.LabelEncoder().fit_transform)
train.movies_emb_columns = train.movies_emb_columns + columns

In [ ]:
# One Hot Encode users
columns = ['Gender', 'Age', 'Occupation']
ohe = preprocessing.OneHotEncoder(categories='auto', sparse=False, dtype='uint8')
ohe.fit(train.ratings[columns])
train.ratings = pd.concat([train.ratings, pd.DataFrame(data=ohe.transform(train.ratings[columns]), columns=ohe.get_feature_names_out(columns))], axis=1)
train.users_ohe_columns = ohe.get_feature_names_out(columns)

assert train.ratings[train.users_ohe_columns].max().max()<=1, 'Error with ohe columns'

In [ ]:
# One Hot Encode movies
genres = ["genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "FilmNoir", "Horror",
    "Musical", "Mystery", "Romance", "SciFi", "Thriller", "War", "Western"]

for genre in genres:
    genre = genre.replace('-', '')
    column = str(genre)
    train.ratings[column] = train.ratings['Genre'].apply(lambda x: 1 if genre in x else 0)
    train.movies_ohe_columns.append(column)

assert train.ratings[train.movies_ohe_columns].max().max()<=1, 'Error with ohe columns'

In [ ]:
int_genres_gender = ""
for genre in train.movies_ohe_columns:
    int_genres_gender = int_genres_gender + '+' +genre + ':Gender'

int_genres_age = ""
for genre in train.movies_ohe_columns:
    int_genres_age = int_genres_age + '+' + genre + ':Age'

interact = patsy.dmatrix("0 + Gender:Age + Gender:Occupation + Age:Occupation"+int_genres_gender+int_genres_age, data=train.ratings.astype('object'), return_type='dataframe').astype('int8')
interact = interact.astype('uint8')
train.ratings = pd.concat([train.ratings, interact], axis=1)
train.interact_columns = interact.columns

In [ ]:
interact


,Gender[0]:Age[0],Gender[1]:Age[0],Gender[0]:Age[1],Gender[1]:Age[1],Gender[0]:Age[2],Gender[1]:Age[2],Gender[0]:Age[3],Gender[1]:Age[3],Gender[0]:Age[4],Gender[1]:Age[4],...,War[T.1]:Age[T.3],War[T.1]:Age[T.4],War[T.1]:Age[T.5],War[T.1]:Age[T.6],Western[T.1]:Age[T.1],Western[T.1]:Age[T.2],Western[T.1]:Age[T.3],Western[T.1]:Age[T.4],Western[T.1]:Age[T.5],Western[T.1]:Age[T.6]
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
99996,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
99997,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
99998,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Drop unused columns
train.movies.drop(['Title', 'Genre'], inplace=True, axis=1)
train.ratings.drop(['Title', 'Genre', 'Zipcode'], inplace=True, axis=1)

In [ ]:
train.to_tensor()

### DataLoaders

In [ ]:
# Split
train_size = int(0.8 * len(train))
test_size = len(train) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(train, [train_size, test_size])

# Create dataloaders
dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(train_dataset, batch_size=4096, shuffle=True)
dataloaders['valid'] = torch.utils.data.DataLoader(valid_dataset, batch_size=4096, shuffle=True)

### Define Pytorch models

In [ ]:
class deepnwide(nn.Module):

    def __init__(self, users_emb, movies_emb, users_ohe, movies_ohe, interact, size_emb, y_range, dropout, linear_size= 500):
        super().__init__()

        self.name = 'deepnwide'
        self.y_range = y_range

        # wide part

        # deep
        self.emb_UserID = nn.Embedding(len(torch.unique(users_emb[:, 0])), size_emb)
        self.emb_UserID.weight.data.uniform_(-.01, .01)
        self.emb_Gender = nn.Embedding(len(torch.unique(users_emb[:, 1])), size_emb)
        self.emb_Gender.weight.data.uniform_(-.01, .01)
        self.emb_Age = nn.Embedding(len(torch.unique(users_emb[:, 2])), size_emb)
        self.emb_Age.weight.data.uniform_(-.01, .01)
        self.emb_Occupation = nn.Embedding(len(torch.unique(users_emb[:, 3])), size_emb)
        self.emb_Occupation.weight.data.uniform_(-.01, .01)
        self.emb_MovieID = nn.Embedding(len(torch.unique(movies_emb[:, 0])), size_emb)
        self.emb_MovieID.weight.data.uniform_(-.01, .01)

        # hidden layers
        self.h1 = nn.Linear(5 * size_emb, linear_size)
        self.h2 = nn.Linear(linear_size, linear_size)
        self.h3 = nn.Linear(linear_size, linear_size)

        # Dropout layers
        self.dropout1 = nn.Dropout(p=dropout)
        self.dropout2 = nn.Dropout(p=dropout)
        self.dropout3 = nn.Dropout(p=dropout)

        # final dense layer
        self.last_layer = nn.Linear((interact.shape[1]) + (movies_ohe.shape[1]) + (linear_size), 1)


    def forward(self, X):
        # Assign data
        user_emb = X[0]
        user_ohe = X[1]
        movie_emb = X[2]
        movie_ohe = X[3]
        interact = X[4]

        UserID = user_emb[:, 0]
        Gender = user_emb[:, 1]
        Age = user_emb[:, 2]
        Occupation = user_emb[:, 3]
        MovieID = movie_emb[:, 0]

        UserID = self.emb_UserID(UserID)
        Gender = self.emb_Gender(Gender)
        Age = self.emb_Age(Age)
        Occupation = self.emb_Occupation(Occupation)
        MovieID = self.emb_MovieID(MovieID)

        emb = torch.cat([UserID,
                         Age,
                         Gender,
                         Occupation,
                         MovieID],
                         dim=1)

        emb = F.relu(self.dropout1(self.h1(emb)))
        emb = F.relu(self.dropout2(self.h2(emb)))
        emb = F.relu(self.dropout3(self.h3(emb)))

        result = self.last_layer(torch.cat([interact.float(), movie_ohe.float(), emb.float()], dim=1))

        return (torch.sigmoid(result) * (self.y_range[1]-self.y_range[0]) + self.y_range[0]).squeeze()


model = deepnwide(train.users_emb, train.movies_emb, train.users_ohe, train.movies_ohe, train.interact, 60, train.y_range, 0.5, 100)
model.to(device)
print(model)

deepnwide(
  (emb_UserID): Embedding(943, 60)
  (emb_Gender): Embedding(2, 60)
  (emb_Age): Embedding(7, 60)
  (emb_Occupation): Embedding(21, 60)
  (emb_MovieID): Embedding(1682, 60)
  (h1): Linear(in_features=300, out_features=100, bias=True)
  (h2): Linear(in_features=100, out_features=100, bias=True)
  (h3): Linear(in_features=100, out_features=100, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (dropout3): Dropout(p=0.5, inplace=False)
  (last_layer): Linear(in_features=445, out_features=1, bias=True)
)


In [27]:
class twoembeds(torch.nn.Module):

    def __init__(self, size_emb, y_range):
        super().__init__()

        # set name of model
        self.name = 'twoembeds'
        self.y_range = y_range

        # User and movie embeddings
        self.emb_UserID = nn.Embedding(train.nusers, size_emb)
        self.emb_MovieID = nn.Embedding(train.nmovies, size_emb)
        self.emb_UserID.weight.data.uniform_(-.01, .01)
        self.emb_MovieID.weight.data.uniform_(-.01, .01)

        # User and movie embeddings weights
        self.emb_UserID_b = nn.Embedding(train.nusers, 1)
        self.emb_MovieID_b = nn.Embedding(train.nmovies, 1)
        self.emb_UserID_b.weight.data.uniform_(-.01, .01)
        self.emb_MovieID_b.weight.data.uniform_(-.01, .01)


    def forward(self, X):
        user_emb = X[0]
        user_ohe = X[1]
        movie_emb = X[2]
        movie_ohe = X[3]
        interact = X[4]

        UserID = user_emb[:, 0]
        MovieID = movie_emb[:, 0]

        user_emb = self.emb_UserID(UserID)
        movie_emb = self.emb_MovieID(MovieID)

        mult = (user_emb * movie_emb).sum(1)

        # add bias
        multb = mult + self.emb_UserID_b(UserID).squeeze() + self.emb_MovieID_b(MovieID).squeeze()

        multb = multb.float()

        return (torch.sigmoid(multb) * (self.y_range[1]-self.y_range[0]) + self.y_range[0]).squeeze()

        return multb


model = twoembeds(15, train.y_range)
model.to(device)
print(model)

twoembeds(
  (emb_UserID): Embedding(943, 15)
  (emb_MovieID): Embedding(1682, 15)
  (emb_UserID_b): Embedding(943, 1)
  (emb_MovieID_b): Embedding(1682, 1)
)


In [ ]:
class ncf(torch.nn.Module):

    def __init__(self, users_emb, movies_emb, users_ohe, movies_ohe, interact, size_emb, dropout, linear_size, y_range):
        super().__init__()

        # set name of model
        self.name = 'ncf'
        self.y_range = y_range

        ### GMF part
        # user embeddings
        self.gmf_embuserid = nn.Embedding(len(torch.unique(users_emb[:, 0])), size_emb)
        self.gmf_embuserid.weight.data.uniform_(-.01, .01)
        self.gmf_embgender = nn.Embedding(len(torch.unique(users_emb[:, 1])), size_emb)
        self.gmf_embgender.weight.data.uniform_(-.01, .01)
        self.gmf_embage = nn.Embedding(len(torch.unique(users_emb[:, 2])), size_emb)
        self.gmf_embage.weight.data.uniform_(-.01, .01)
        self.gmf_embocc = nn.Embedding(len(torch.unique(users_emb[:, 3])), size_emb)
        self.gmf_embocc.weight.data.uniform_(-.01, .01)
        # movie embeddings
        self.gmf_embmovieid = nn.Embedding(len(torch.unique(movies_emb[:, 0])), size_emb*4-len(train.movies_ohe_columns))
        self.gmf_embmovieid.weight.data.uniform_(-.01, .01)


        ### MLP part
        # user embeddings
        self.mlp_embuserid = nn.Embedding(len(torch.unique(users_emb[:, 0])), size_emb)
        self.mlp_embuserid.weight.data.uniform_(-.01, .01)
        self.mlp_embgender = nn.Embedding(len(torch.unique(users_emb[:, 1])), size_emb)
        self.mlp_embgender.weight.data.uniform_(-.01, .01)
        self.mlp_embage = nn.Embedding(len(torch.unique(users_emb[:, 2])), size_emb)
        self.mlp_embage.weight.data.uniform_(-.01, .01)
        self.mlp_embocc = nn.Embedding(len(torch.unique(users_emb[:, 3])), size_emb)
        self.mlp_embocc.weight.data.uniform_(-.01, .01)
        # movie embeddings
        self.mlp_embmovieid = nn.Embedding(len(torch.unique(movies_emb[:, 0])), size_emb)
        self.mlp_embmovieid.weight.data.uniform_(-.01, .01)
        # hidden layers
        self.h1 = nn.Linear(5*size_emb+len(train.movies_ohe_columns), linear_size)
        self.h2 = nn.Linear(linear_size, int(linear_size/2))
        #self.h3 = nn.Linear(linear_size, linear_size)
        # Dropout layers
        self.dropout1 = nn.Dropout(p=dropout)
        self.dropout2 = nn.Dropout(p=dropout)
        #self.dropout3 = nn.Dropout(p=dropout)

        # final dense layer
        self.last_layer = nn.Linear(size_emb*4+int(linear_size/2), 1)

    def forward(self, X):
        user_emb = X[0]
        user_ohe = X[1]
        movie_emb = X[2]
        movie_ohe = X[3]
        interact = X[4]

        UserID = user_emb[:, 0]
        Gender = user_emb[:, 1]
        Age = user_emb[:, 2]
        Occupation = user_emb[:, 3]
        MovieID = movie_emb[:, 0]

        # GMF part
        gmf_embuserid = self.gmf_embuserid(UserID)
        gmf_embgender = self.gmf_embgender(Gender)
        gmf_embage = self.gmf_embage(Age)
        gmf_embocc = self.gmf_embocc(Occupation)
        gmf_embmovieid = self.gmf_embmovieid(MovieID)

        gmf_user_vector = torch.cat([gmf_embuserid,
                                    gmf_embgender,
                                    gmf_embage,
                                    gmf_embocc],
                                    dim=1)

        gmf_movie_vector = torch.cat([gmf_embmovieid, movie_ohe], 1)

        gmf_vector = (gmf_user_vector * gmf_movie_vector)


        # MLP part
        mlp_embuserid = self.mlp_embuserid(UserID)
        mlp_embgender = self.mlp_embgender(Gender)
        mlp_embage = self.mlp_embage(Age)
        mlp_embocc = self.mlp_embocc(Occupation)
        mlp_movieid = self.mlp_embmovieid(MovieID)

        mlp_vector = torch.cat([mlp_embuserid,
                                mlp_embgender,
                                mlp_embage,
                                mlp_embocc,
                                mlp_movieid,
                                movie_ohe],
                                dim=1)
        mlp_vector = F.relu(self.dropout1(self.h1(mlp_vector)))
        mlp_vector = F.relu(self.dropout2(self.h2(mlp_vector)))
        #mlp_vector = F.relu(self.dropout3(self.h3(mlp_vector)))

        # Fusion
        result = torch.cat([gmf_vector, mlp_vector], dim=1)
        result = self.last_layer(result)

        #return (torch.sigmoid(result) * (5-1) + 1).squeeze
        return (torch.sigmoid(result) * (self.y_range[1]-self.y_range[0]) + self.y_range[0]).squeeze()


model = ncf(train.users_emb, train.movies_emb, train.users_ohe, train.movies_ohe, train.interact, 60, 0.5, 200, train.y_range)
model.to(device)
print(model)

ncf(
  (gmf_embuserid): Embedding(943, 60)
  (gmf_embgender): Embedding(2, 60)
  (gmf_embage): Embedding(7, 60)
  (gmf_embocc): Embedding(21, 60)
  (gmf_embmovieid): Embedding(1682, 221)
  (mlp_embuserid): Embedding(943, 60)
  (mlp_embgender): Embedding(2, 60)
  (mlp_embage): Embedding(7, 60)
  (mlp_embocc): Embedding(21, 60)
  (mlp_embmovieid): Embedding(1682, 60)
  (h1): Linear(in_features=319, out_features=200, bias=True)
  (h2): Linear(in_features=200, out_features=100, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (last_layer): Linear(in_features=340, out_features=1, bias=True)
)


### Skorch callbacks

In [ ]:
# Earlystopping callback
earlystopping = EarlyStopping(monitor='valid_loss', patience=10, threshold=0.001)

In [ ]:
# RMSE callback
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

def rmseloss(y_true, y_pred):
    #return f1_score(y_true, y_pred)
    return np.sqrt(mean_squared_error(y_true, y_pred))

def precision(y_true, y_pred):
    y_true_binary = (y_true >= 4).astype(int)
    y_pred_binary = (y_pred >= 4).astype(int)

    # Calculate Precision and Recall
    precision_scoree = precision_score(y_true_binary, y_pred_binary)
    return precision_scoree

def recall(y_true, y_pred):
    y_true_binary = (y_true >= 4).astype(int)
    y_pred_binary = (y_pred >= 4).astype(int)

    # Calculate Precision and Recall
    recall_scoree = recall_score(y_true_binary, y_pred_binary)
    return recall_scoree

def f1(y_true, y_pred):
    y_true_binary = (y_true >= 4).astype(int)
    y_pred_binary = (y_pred >= 4).astype(int)

    # Calculate F1 score
    f1_scoree = f1_score(y_true_binary, y_pred_binary)
    return f1_scoree

rmse_scorer = make_scorer(rmseloss)
precision_scorer = make_scorer(precision)
recall_scorer = make_scorer(recall)
f1_scorer = make_scorer(f1)

epoch_rmse = EpochScoring(rmse_scorer, name='rmse_score', lower_is_better=True)
epoch_precision = EpochScoring(precision_scorer, name='precision', lower_is_better= False)
epoch_recall = EpochScoring(recall_scorer, name='recall', lower_is_better= False)
epoch_f1 = EpochScoring(f1_scorer, name='f1', lower_is_better= False)


In [ ]:
# Checkpoint callback
checkpoint = Checkpoint(monitor='rmse_score_best', f_params='params.pt', f_optimizer='optimizer.pt', f_history='history.json', f_pickle='model')

In [ ]:
# Learning rate scheduler callback
lr_scheduler = LRScheduler(policy="StepLR", step_size=7, gamma=0.1)

### Neural Collaborative Filtering

#### Manually specify hyperparamers

In [ ]:
#86.57
ncfnet = NeuralNet(
    ncf,
    module__users_emb=train.users_emb,
    module__movies_emb=train.movies_emb,
    module__users_ohe=train.users_ohe,
    module__movies_ohe=train.movies_ohe,
    module__interact=train.interact,
    module__size_emb=500,
    module__dropout=0.5,
    module__linear_size=1200,
    module__y_range=train.y_range,#### Manually specify hyperparamers
    max_epochs=50,
    lr=0.001,
    optimizer=torch.optim.Adam,
    criterion=torch.nn.MSELoss,
    device=device,
    iterator_train__batch_size=1024,
    iterator_train__num_workers=0,
    iterator_train__shuffle=True,
    iterator_valid__batch_size=1024,
    train_split=predefined_split(valid_dataset),
    callbacks=[
               earlystopping,
               epoch_rmse,
               epoch_precision,
               epoch_recall,
               epoch_f1,
               checkpoint,
               lr_scheduler,
               #TensorBoard(writer),
               #progressbar
               ]
)

In [ ]:
ncfnet.fit(train_dataset)

  epoch      f1    precision    recall    rmse_score    train_loss    valid_loss    cp      lr     dur
-------  ------  -----------  --------  ------------  ------------  ------------  ----  ------  ------
      1  0.4843       0.8280    0.3423        0.9446        1.0215        0.8923     +  0.0010  3.7076
      2  0.4783       0.8512    0.3326        0.9252        0.8626        0.8559     +  0.0010  4.0770
      3  0.5348       0.8506    0.3900        0.9142        0.7350        0.8357     +  0.0010  4.0622
      4  0.5927       0.8345    0.4595        0.9130        0.5401        0.8336     +  0.0010  4.2478
      5  0.5978       0.8247    0.4688        0.9233        0.3581        0.8525        0.0010  4.1074
      6  0.6014       0.8183    0.4754        0.9312        0.2277        0.8672        0.0010  4.0494
      7  0.6254       0.8030    0.5121        0.9401        0.1508        0.8838        0.0010  3.7129
      8  0.6015       0.8075    0.4792        0.9410        0.1064       

<class 'skorch.net.NeuralNet'>[initialized](
  module_=ncf(
    (gmf_embuserid): Embedding(943, 500)
    (gmf_embgender): Embedding(2, 500)
    (gmf_embage): Embedding(7, 500)
    (gmf_embocc): Embedding(21, 500)
    (gmf_embmovieid): Embedding(1682, 1981)
    (mlp_embuserid): Embedding(943, 500)
    (mlp_embgender): Embedding(2, 500)
    (mlp_embage): Embedding(7, 500)
    (mlp_embocc): Embedding(21, 500)
    (mlp_embmovieid): Embedding(1682, 500)
    (h1): Linear(in_features=2519, out_features=1200, bias=True)
    (h2): Linear(in_features=1200, out_features=600, bias=True)
    (dropout1): Dropout(p=0.5, inplace=False)
    (dropout2): Dropout(p=0.5, inplace=False)
    (last_layer): Linear(in_features=2600, out_features=1, bias=True)
  ),
)

#### GridSearchCV

In [ ]:
params = {
    'lr': [0.01],
    'module__size_emb': [30, 60, 120],
    'module__dropout': [0.3],
    'module__linear_size': [200, 400]
}
gs = GridSearchCV(ncfnet,
                  params,
                  verbose=50,
                  refit=False,
                  pre_dispatch=2,
                  n_jobs=1,
                  cv=3,
                  scoring='neg_mean_squared_error')

X_ds = SliceDataset(train, idx=0)
y_ds = SliceDataset(train, idx=1)
gs.fit(X_ds, y_ds)

print(gs.best_score_, gs.best_params_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV 1/3; 1/6] START lr=0.01, module__dropout=0.3, module__linear_size=200, module__size_emb=30
  epoch      f1    precision    recall    rmse_score    train_loss    valid_loss    cp      lr     dur
-------  ------  -----------  --------  ------------  ------------  ------------  ----  ------  ------
      1  0.4183       0.8544    0.2770        0.9731        0.9965        0.9469     +  0.0100  7.2017
      2  0.6149       0.9057    0.4654        0.8364        0.7648        0.6995     +  0.0100  7.4854
      3  0.6850       0.9193    0.5458        0.7758        0.4897        0.6018     +  0.0100  5.8225
      4  0.7077       0.9300    0.5711        0.7475        0.3499        0.5587     +  0.0100  7.6369
      5  0.7415       0.9204    0.6208        0.7389        0.2852        0.5460     +  0.0100  6.5246
      6  0.7726       0.9131    0.6696        0.7407        0.2482        0.5487        0.0100  7.8110
      7  0.6949      

### Deep and Wide

#### Manually specify hyperparamers

In [ ]:
deepnwidenet = NeuralNet(
    deepnwide,
    module__users_emb=train.users_emb,
    module__movies_emb=train.movies_emb,
    module__users_ohe=train.users_ohe,
    module__movies_ohe=train.movies_ohe,
    #### Manually specify hyperparamers e=train.movies_ohe,
    module__interact=train.interact,
    module__size_emb=30,
    module__y_range=train.y_range,
    module__dropout=0.2,
    max_epochs=30,
    lr=0.001,
    optimizer=torch.optim.Adam,
    criterion=torch.nn.MSELoss,
    device=device,
    iterator_train__batch_size=1024,
    iterator_train__num_workers=0,
    iterator_train__shuffle=True,
    iterator_valid__batch_size=4096,
    train_split=predefined_split(valid_dataset),
    callbacks=[
               earlystopping,
               epoch_rmse,
               epoch_precision,
               epoch_recall,
               epoch_f1,
               #checkpoint,
               lr_scheduler,
               #TensorBoard(writer),
               #progressbar
               ]
)

In [ ]:
deepnwidenet.fit(train_dataset)

  epoch      f1    precision    recall    rmse_score    train_loss    valid_loss      lr     dur
-------  ------  -----------  --------  ------------  ------------  ------------  ------  ------
      1  0.5067       0.8408    0.3626        0.9640        1.0904        0.9294  0.0010  7.7631
      2  0.5385       0.8312    0.3982        0.9474        0.8823        0.8976  0.0010  2.9171
      3  0.4556       0.8532    0.3107        0.9453        0.8592        0.8936  0.0010  2.8487
      4  0.5504       0.8346    0.4106        0.9405        0.8501        0.8845  0.0010  3.6234
      5  0.4931       0.8494    0.3474        0.9397        0.8375        0.8830  0.0010  3.3253
      6  0.4781       0.8581    0.3313        0.9387        0.8264        0.8811  0.0010  3.0694
      7  0.5384       0.8439    0.3953        0.9366        0.8172        0.8772  0.0010  2.8755
      8  0.5366       0.8480    0.3925        0.9366        0.7887        0.8772  0.0001  3.4009
      9  0.5457       0.8444  

<class 'skorch.net.NeuralNet'>[initialized](
  module_=deepnwide(
    (emb_UserID): Embedding(943, 30)
    (emb_Gender): Embedding(2, 30)
    (emb_Age): Embedding(7, 30)
    (emb_Occupation): Embedding(21, 30)
    (emb_MovieID): Embedding(1682, 30)
    (h1): Linear(in_features=150, out_features=500, bias=True)
    (h2): Linear(in_features=500, out_features=500, bias=True)
    (h3): Linear(in_features=500, out_features=500, bias=True)
    (dropout1): Dropout(p=0.2, inplace=False)
    (dropout2): Dropout(p=0.2, inplace=False)
    (dropout3): Dropout(p=0.2, inplace=False)
    (last_layer): Linear(in_features=845, out_features=1, bias=True)
  ),
)

#### GridsearchCV

In [ ]:
# params = {
#     'lr': [0.001, 0.01],
#     'module__size_emb': [30, 60, 120],
#     'module__dropout': [0.5],
#     'module__linear_size': [400, 500, 600]
# }
params = {
    'lr': [0.01, 0.001],
    'module__size_emb': [30, 60, 120],
    'module__dropout': [0.3, 0.5],
    'module__linear_size': [400, 500, 600]
}
gs = GridSearchCV(deepnwidenet,
                  params,
                  verbose=50,
                  refit=False,
                  #pre_dispatch=8,
                  n_jobs=1,
                  cv=3,
                  scoring='neg_mean_squared_error')

X_ds = SliceDataset(train, idx=0)
y_ds = SliceDataset(train, idx=1)
gs.fit(X_ds, y_ds)

print(gs.best_score_, gs.best_params_)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV 1/3; 1/36] START lr=0.01, module__dropout=0.3, module__linear_size=400, module__size_emb=30
  epoch      f1    precision    recall    rmse_score    train_loss    valid_loss      lr     dur
-------  ------  -----------  --------  ------------  ------------  ------------  ------  ------
      1  0.3928       0.8538    0.2551        0.9929        0.9993        0.9858  0.0100  3.6870
      2  0.4698       0.8389    0.3262        0.9738        0.8905        0.9482  0.0100  3.6836
      3  0.5845       0.8091    0.4575        0.9857        0.8617        0.9717  0.0100  4.2670
      4  0.5128       0.8349    0.3700        0.9646        0.8515        0.9305  0.0100  3.6281
      5  0.6784       0.7743    0.6037        1.0065        0.8368        1.0131  0.0100  3.6015
      6  0.4505       0.8528    0.3061        0.9574        0.8337        0.9165  0.0100  4.4760
      7  0.5798       0.8228    0.4476        0.9528        0.8095

### Two embeddings - Basic matrix factorization

#### Manually specify hyperparamers

In [28]:
twoembedsnet = NeuralNet(
    twoembeds,
    module__size_emb=128,
    module__y_range=train.y_range,
    max_epochs=30,
    lr=0.001,
    optimizer=torch.optim.Adam,
    criterion=torch.nn.MSELoss,
    device=device,
    iterator_train__batch_size=4096,
    iterator_train__num_workers=0,
    iterator_train__shuffle=True,
    iterator_valid__batch_size=4096,
    train_split=predefined_split(valid_dataset),
    callbacks=[earlystopping,
               epoch_rmse,
               epoch_precision,
               epoch_recall,
               epoch_f1,
               #checkpoint,
               lr_scheduler]
)

In [ ]:
twoembedsnet.fit(train_dataset)

  epoch      f1    precision    recall    rmse_score    train_loss    valid_loss      lr     dur
-------  ------  -----------  --------  ------------  ------------  ------------  ------  ------
      1  0.0000       0.0000    0.0000        1.2246        1.5324        1.4996  0.0010  3.9334


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


      2  0.0000       0.0000    0.0000        1.1950        1.4785        1.4281  0.0010  6.3667


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


      3  0.0000       0.0000    0.0000        1.1286        1.3628        1.2738  0.0010  6.8263


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Exception ignored in: <function _xla_gc_callback at 0x7e2ca9ea5900>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


      4  0.0000       0.0000    0.0000        1.0460        1.1728        1.0941  0.0010  4.0623


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


      5  0.1819       0.8932    0.1013        0.9964        1.0072        0.9928  0.0010  4.8329
      6  0.3995       0.8504    0.2611        0.9762        0.9211        0.9530  0.0010  6.7228
      7  0.4543       0.8476    0.3103        0.9645        0.8750        0.9302  0.0010  4.0794
      8  0.4569       0.8480    0.3126        0.9635        0.8496        0.9283  0.0001  4.5573
      9  0.4577       0.8487    0.3134        0.9625        0.8464        0.9265  0.0001  3.8373
     10  0.4592       0.8484    0.3148        0.9616        0.8434        0.9247  0.0001  3.8165
     11  0.4598       0.8479    0.3154        0.9608        0.8404        0.9231  0.0001  4.7428
     12  0.4611       0.8474    0.3167        0.9600        0.8375        0.9215  0.0001  3.7458
     13  0.4631       0.8472    0.3187        0.9592        0.8348        0.9200  0.0001  3.9336
     14  0.4649       0.8467    0.3204        0.9584        0.8320        0.9185  0.0001  4.9544
     15  0.4649       0.8465  

<class 'skorch.net.NeuralNet'>[initialized](
  module_=twoembeds(
    (emb_UserID): Embedding(943, 128)
    (emb_MovieID): Embedding(1682, 128)
    (emb_UserID_b): Embedding(943, 1)
    (emb_MovieID_b): Embedding(1682, 1)
  ),
)

#### GridSearchCV

In [ ]:
params = {
    'lr': [0.01],
    'module__size_emb': [600, 700, 800, 900, 1000]
}
gs = GridSearchCV(twoembedsnet,
                  params,
                  verbose=50,
                  refit=False,
                  #pre_dispatch=8,
                  #n_jobs=8,
                  cv=3,
                  scoring='neg_mean_squared_error')

X_ds = SliceDataset(train, idx=0)
y_ds = SliceDataset(train, idx=1)
gs.fit(X_ds, y_ds)

print(gs.best_score_, gs.best_params_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV 1/3; 1/5] START lr=0.01, module__size_emb=600...............................
  epoch      f1    precision    recall    rmse_score    train_loss    valid_loss      lr      dur
-------  ------  -----------  --------  ------------  ------------  ------------  ------  -------
      1  0.5491       0.8208    0.4125        1.0011        1.2399        1.0023  0.0100  10.2454
      2  0.5246       0.9604    0.3608        0.8418        0.6454        0.7087  0.0100  7.5932
      3  0.6880       0.9882    0.5277        0.7527        0.2842        0.5665  0.0100  7.1685
      4  0.7137       0.9945    0.5566        0.7242        0.1335        0.5245  0.0100  7.5094
      5  0.7032       0.9965    0.5432        0.7112        0.0844        0.5058  0.0100  6.5538
      6  0.6918       0.9958    0.5300        0.7068        0.0638        0.4996  0.0100  7.9443
      7  0.6831       0.9953    0.5199        0.7052        0.0549        0.4973

### Benchmark with scikit-surprise SVD algorithm

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163765 sha256=d9f3d8dac7c135ab0e4772500fbfaa880c694df7ce5e0e381c317e20daad7ff2
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import NormalPredictor
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split, KFold

In [ ]:
train[dataloaders['train'].dataset.indices][1]

tensor([4., 4., 4.,  ..., 2., 2., 2.])

In [ ]:
user = train[dataloaders['train'].dataset.indices][0][0][:, 0].data.numpy()
movie = train[dataloaders['train'].dataset.indices][0][2][:, 0].data.numpy()
y = train[dataloaders['train'].dataset.indices][1].data.numpy()
df = pd.DataFrame({'user': user, 'movie': movie, 'y': y})
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user', 'movie', 'y']], reader)

In [ ]:
data = Dataset.load_from_df(train.ratings.loc[dataloaders['train'].dataset.indices, ['UserID', 'MovieID', 'Rating']], reader)

In [ ]:
a = train.ratings.loc[dataloaders['train'].dataset.indices, ['UserID', 'MovieID', 'Rating']]
b = pd.DataFrame({'UserID': user, 'MovieID': movie, 'Rating': y})

In [ ]:
trainset, testset = train_test_split(data, test_size=.25)

algo = SVD()

algo.fit(trainset)
predictions = algo.test(testset)

#RMSE
accuracy.rmse(predictions)

RMSE: 0.9532


0.9532214288290356